# CLX Asset Classification (Supervised)

## Authors
- Eli Fajardo (NVIDIA)
- Görkem Batmaz (NVIDIA)
- Bhargav Suryadevara (NVIDIA)


## Table of Contents 
* Introduction
* Dataset
* Reading in the datasets
* Training and inference
* References

# Introduction

In this notebook, we will show how to predict the function of a server with Windows Event Logs using cudf, cuml and pytorch. The machines are labeled as DC, SQL, WEB, DHCP, MAIL and SAP. The dependent variable will be the type of the machine. The features are selected from Windows Event Logs which is in a tabular format. This is a first step to learn the behaviours of certain types of machines in data-centres by classifying them probabilistically. It could help to detect unusual behaviour in a data-centre. For example, some compromised computers might be acting as web/database servers but with their original tag. 

This work could be expanded by using different log types or different events from the machines as features to improve accuracy. Various labels can be selected to cover different types of machines or data-centres.

## Library imports

In [1]:
from clx.analytics.asset_classification import AssetClassification
import cudf
from cuml.preprocessing import train_test_split
from cuml.preprocessing import LabelEncoder
import torch
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

## Initialize variables

10000 is chosen as the batch size to optimise the performance for this dataset. It can be changed depending on the data loading mechanism or the setup used. 

EPOCH should also be adjusted depending on convergence for a specific dataset. 

label_col indicates the total number of features used plus the dependent variable. Feature names are listed below.

In [2]:
batch_size = 10000
label_col = '19'
epochs = 15

In [3]:
ac = AssetClassification()

## Read the dataset into a GPU dataframe with `cudf.read_csv()` 

The original data had many other fields. Many of them were either static or mostly blank. After filtering those, there were 18 meaningful columns left. In this notebook we use a fake continuous feature to show the inclusion of continuous features too. When you are using raw data the cell below need to be uncommented

In [4]:
# win_events_gdf = cudf.read_csv("raw_features_and_labels.csv")

```
win_events_gdf.dtypes

eventcode                                                       int64
keywords                                                       object
privileges                                                     object
message                                                        object
sourcename                                                     object
taskcategory                                                   object
account_for_which_logon_failed_account_domain                  object
detailed_authentication_information_authentication_package     object
detailed_authentication_information_key_length                float64
detailed_authentication_information_logon_process              object
detailed_authentication_information_package_name_ntlm_only     object
logon_type                                                    float64
network_information_workstation_name                           object
new_logon_security_id                                          object
impersonation_level                                            object
network_information_protocol                                  float64
network_information_direction                                  object
filter_information_layer_name                                  object
cont1                                                           int64
label                                                          object
dtype: object
```

### Define categorical and continuous feature columns.

In [5]:
cat_cols = [
    "eventcode",
    "keywords",
    "privileges",
    "message",
    "sourcename",
    "taskcategory",
    "account_for_which_logon_failed_account_domain",
    "detailed_authentication_information_authentication_package",
    "detailed_authentication_information_key_length",
    "detailed_authentication_information_logon_process",
    "detailed_authentication_information_package_name_ntlm_only",
    "logon_type",
    "network_information_workstation_name",
    "new_logon_security_id",
    "impersonation_level",
    "network_information_protocol",
    "network_information_direction",
    "filter_information_layer_name",
    "label"
]

In [6]:
cont_cols = [
    "cont1"
]

The following are functions used to preprocess categorical and continuous feature columns. This can very depending on what best fits your application and data.

In [7]:
def categorize_columns(cat_gdf):
    for col in cat_gdf.columns:
        cat_gdf[col] = cat_gdf[col].astype('str')
        cat_gdf[col] = cat_gdf[col].fillna("NA")
        cat_gdf[col] = LabelEncoder().fit_transform(cat_gdf[col])
        cat_gdf[col] = cat_gdf[col].astype('int16')
        
    return cat_gdf

In [8]:
def normalize_conts(cont_gdf):
    means, stds = (cont_gdf.mean(0), cont_gdf.std(ddof=0))
    cont_gdf = (cont_gdf - means) / stds
    
    return cont_gdf

Preprocessing steps below are not executed in this notebook, because we release already preprocessed data.

In [9]:
#win_events_gdf[cat_cols] = categorize_columns(win_events_gdf[cat_cols])

In [10]:
#win_events_gdf[cont_cols] = normalize_conts(win_events_gdf[cont_cols])

Read Windows Event data already preprocessed by above steps

In [11]:
win_events_gdf = cudf.read_csv("win_events_features_preproc.csv")

In [12]:
win_events_gdf.head()

,eventcode,keywords,privileges,message,sourcename,taskcategory,account_for_which_logon_failed_account_domain,detailed_authentication_information_authentication_package,detailed_authentication_information_key_length,detailed_authentication_information_logon_process,detailed_authentication_information_package_name_ntlm_only,logon_type,network_information_workstation_name,new_logon_security_id,impersonation_level,network_information_protocol,network_information_direction,filter_information_layer_name,cont1,label
0,0,1,0,15,0,4,22,0,0,5,0,1,932,38,3,6,1,1,-1.732030,1
1,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731988,0
2,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731945,0
3,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731903,0
4,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731861,0


### Split the dataset into training and test sets using cuML `train_test_split` function
Column 19 contains the ground truth about each machine's function that the logs come from. i.e. DC, SQL, WEB, DHCP, MAIL and SAP. Hence it will be used as a label.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(win_events_gdf, "label", train_size=0.9)
X_train["label"] = Y_train

In [14]:
X_train.head()

,eventcode,keywords,privileges,message,sourcename,taskcategory,account_for_which_logon_failed_account_domain,detailed_authentication_information_authentication_package,detailed_authentication_information_key_length,detailed_authentication_information_logon_process,detailed_authentication_information_package_name_ntlm_only,logon_type,network_information_workstation_name,new_logon_security_id,impersonation_level,network_information_protocol,network_information_direction,filter_information_layer_name,cont1,label
30198,0,1,0,15,0,4,22,0,0,5,0,1,932,2107,2,6,1,1,-0.457679,2
48385,0,1,0,15,0,4,22,0,0,5,0,1,932,1332,2,6,1,1,0.309810,2
79643,0,1,0,15,0,4,22,4,1,7,3,1,932,7980,3,6,1,1,1.628893,1
26051,0,1,0,15,0,4,22,1,0,0,0,1,107,7919,2,6,1,1,-0.632681,0
77438,0,1,0,15,0,4,22,0,0,5,0,1,932,2115,2,6,1,1,1.535843,2


In [15]:
Y_train.unique()

0    0
1    1
2    2
3    3
4    4
5    5
Name: label, dtype: int64

### Print Labels
Making sure the test set contains all labels

In [16]:
Y_test.unique()

0    0
1    1
2    2
3    3
4    4
5    5
Name: label, dtype: int64

## Training 

Asset Classification training uses the fastai tabular model. More details can be found at https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6

Feature columns will be embedded so that they can be used as categorical values. The limit can be changed depending on the accuracy of the dataset.

Adam is the optimizer used in the training process; it is popular because it produces good results in various tasks. In its paper, computing the first and the second moment estimates and updating the parameters are summarized as follows

$$\alpha_{t}=\alpha \cdot \sqrt{1-\beta_{2}^{t}} /\left(1-\beta_{1}^{t}\right)$$

More detailson Adam can be found at https://arxiv.org/pdf/1412.6980.pdf

We have found that the way we partition the dataframes with a 10000 batch size gives us the optimum data loading capability. The **batch_size** argument can be adjusted for different sizes of datasets.

In [17]:
cat_cols.remove("label")
ac.train_model(X_train, cat_cols, cont_cols, "label", batch_size, epochs, lr=0.01, wd=0.0)

/opt/conda/envs/rapids/lib/python3.7/site-packages/cudf/io/dlpack.py:74: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return libdlpack.to_dlpack(gdf_cols)


training loss:  1.4238278804552063
valid loss 1.028 and accuracy 0.672
training loss:  0.9197047214713447
valid loss 0.764 and accuracy 0.733
training loss:  0.7048702411719668
valid loss 0.619 and accuracy 0.805
training loss:  0.5750047987891256
valid loss 0.521 and accuracy 0.836
training loss:  0.49577216114918043
valid loss 0.463 and accuracy 0.853
training loss:  0.44185820460441433
valid loss 0.414 and accuracy 0.872
training loss:  0.39902660612105045
valid loss 0.377 and accuracy 0.888
training loss:  0.3620645587339172
valid loss 0.345 and accuracy 0.893
training loss:  0.3338748477974746
valid loss 0.321 and accuracy 0.900
training loss:  0.3105638655056456
valid loss 0.303 and accuracy 0.905
training loss:  0.2919324446411333
valid loss 0.288 and accuracy 0.907
training loss:  0.27697643307501346
valid loss 0.277 and accuracy 0.912
training loss:  0.2654482530163635
valid loss 0.267 and accuracy 0.915
training loss:  0.25572876083978335
valid loss 0.260 and accuracy 0.917
t

## Evaluation

In [18]:
pred_results = ac.predict(X_test).to_array()
true_results = Y_test.to_array()

In [19]:
ac.predict(X_test)

0       0
1       0
2       0
3       0
4       0
       ..
8204    0
8205    0
8206    0
8207    2
8208    2
Length: 8209, dtype: int64

In [20]:
type(ac.predict(X_test))

cudf.core.series.Series

In [21]:
f1_score_ = f1_score(pred_results, true_results, average='micro')
print('micro F1 score: %s'%(f1_score_))

micro F1 score: 0.9160677305396516


In [22]:
torch.cuda.empty_cache()

In [23]:
labels = ["DC","DHCP","MAIL","SAP","SQL","WEB"]
a = confusion_matrix(true_results, pred_results)

In [24]:
pd.DataFrame(a, index=labels, columns=labels)

,DC,DHCP,MAIL,SAP,SQL,WEB
DC,3459,22,19,7,62,7
DHCP,105,675,1,1,12,0
MAIL,12,1,2582,10,13,0
SAP,26,0,5,148,5,0
SQL,235,0,10,22,612,23
WEB,56,0,0,2,33,44


The confusion matrix shows that some machines' function can be predicted really well, whereas some of them need more tuning or more features. This work can be improved and expanded to cover individual data-centres to create a realistic map of the network using ML by not just relying on the naming conventions. It could also help to detect more prominent scale anomalies like multiple machines, not acting per their tag.

## References:
* https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6
* https://jovian.ml/aakashns/04-feedforward-nn
* https://www.kaggle.com/dienhoa/reverse-tabular-module-of-fast-ai-v1
* https://github.com/fastai/fastai/blob/master/fastai/layers.py#L44